<a href="https://colab.research.google.com/github/tummalapallimurali/GenAI/blob/main/BERT_Transfomer_Finetuning_Amazon_Alexa_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

reviews = pd.read_csv("/content/amazon_alexa.tsv",  sep="\t")
pd.set_option("display.max_rows", 5)

print("Setup complete.")

reviews

Setup complete.


,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
...,...,...,...,...,...
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1
3149,4,29-Jul-18,Black Dot,Good,1


# Multi-label classification to classify Given Text and Variation - classify the rating from review for a product.

In [ ]:
# append variation and reviews and create in a list
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Check if text is a string before applying lower()
    if isinstance(text, str):
        doc = nlp(text.lower())  # Convert to lowercase
        tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]  # Lemmatization and stopword removal
        return " ".join(tokens)
    else:
        # Handle non-string values (e.g., NaN)
        return ""  # Or any other appropriate value

# Apply preprocessing to the dataset
reviews['cleaned_reviews'] = reviews['verified_reviews'].apply(preprocess_text)

#

In [ ]:
reviews

,rating,date,variation,verified_reviews,feedback,cleaned_reviews
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1,love echo
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1,love
...,...,...,...,...,...,...
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1,complaint sound quality great use command dot ...
3149,4,29-Jul-18,Black Dot,Good,1,good


In [ ]:
reviews

,rating,date,variation,verified_reviews,feedback,cleaned_reviews,t5_input,t5_output
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1,love echo,rate the review: love echo,5
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1,love,rate the review: love,5
...,...,...,...,...,...,...,...,...
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1,complaint sound quality great use command dot ...,rate the review: complaint sound quality great...,5
3149,4,29-Jul-18,Black Dot,Good,1,good,rate the review: good,4


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

inputs = reviews['t5_input'].tolist()
targets = reviews['t5_output'].tolist()

inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=64, return_tensors="pt")
#targets = tokenizer(targets, padding="max_length", truncation=True, max_length=4, return_tensors="pt")

input_ids, attention_mask = inputs.input_ids.squeeze(), inputs.attention_mask.squeeze()
labels = torch.tensor(reviews['rating'].astype(int).values -1)

import torch

model = AutoModelForSequenceClassification.from_pretrained("t5-base", num_labels=5)

train_dataset = torch.utils.data.TensorDataset(input_ids, attention_mask, labels)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

# fine tuning

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)



Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

for epoch in range(3):
    model.train()
    for batch in train_dataloader:
        #batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} completed.")

In [ ]:
# Convert rating to string for text-to-text format
# append variation and reviews and create in a list
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Check if text is a string before applying lower()
    if isinstance(text, str):
        doc = nlp(text.lower())  # Convert to lowercase
        tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]  # Lemmatization and stopword removal
        return " ".join(tokens)
    else:
        # Handle non-string values (e.g., NaN)
        return ""  # Or any other appropriate value

# Apply preprocessing to the dataset
reviews['cleaned_reviews'] = reviews['verified_reviews'].apply(preprocess_text)

reviews['rating'] = reviews['rating'].astype(str)

# Create input-output pairs
reviews['t5_input'] = "rate the review: " + reviews['cleaned_reviews']
reviews['t5_output'] = reviews['rating']


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import torch

class ReviewDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):


        self.inputs = df['t5_input'].tolist()
        self.targets = df['t5_output'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        target_text = self.targets[idx]

        inputs = self.tokenizer(
            input_text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        targets = self.tokenizer(
            target_text,
            max_length=2,  # Output is a single number (rating)
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze()
        }

# Initialize T5 and Tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Create Dataset and DataLoader
train_dataset = ReviewDataset(reviews, tokenizer, max_len=128)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Training Loop
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # Number of epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1, Loss: 0.720724880695343
Epoch 2, Loss: 0.2602863907814026
Epoch 3, Loss: 0.3600456118583679


In [ ]:
def predict_rating(review):
    input_text = "rate the review: " + preprocess_text(review)
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).input_ids.to(device)
    output = model.generate(input_ids)
    return tokenizer.decode(output[0], skip_special_tokens=True)


comments=["it's great for music, a little hard to figure out how it works",
          "I bought an echo dot that had been refurbished but its stops whatever function its performing quite often, doesn't wake up every time, randomly turns on but does nothing. Do not buy a refurbished Echo dot.",
          "I am not super impressed with Alexa. When my Prime lapsed, she wouldn't play anything. She isn't smart enough to differentiate among spotify accounts so we can't use it for that either. She randomly speaks up when nobody is talking to her. Just today I unplugged her...not sure I'll ever use my Alexa again.",
          "Without having a cellphone, I cannot use many of her features. I have an iPad but do not see that of any use.  It IS a great alarm.  If u r almost deaf, you can hear her alarm in the bedroom from out in the living room, so that is reason enough to keep her.It is fun to ask random questions to hear her response.  She does not seem to be very smartbon politics yet."]

# Example Prediction

for comment in comments:
  print(predict_rating(comment))


5
5
5
5


## Model 2

In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

reviews = pd.read_csv("/content/amazon_alexa.tsv",  sep="\t")
pd.set_option("display.max_rows", 5)

print("Setup complete.")

reviews


sentences = reviews['verified_reviews'].values

sentences = ["[CLS]" + str(sentence) + "[SEP]" for sentence in sentences]

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

tokenized_text = [tokenizer.tokenize(sent) for sent in sentences]

labels = reviews['feedback'].values

print(tokenized_text[1])


Setup complete.


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (549 > 512). Running this sequence through the model will result in indexing errors


['[CLS]', 'loved', 'it', '!', '[SEP]']


In [ ]:
print(tokenized_text[19])

['[CLS]', 'i', 'liked', 'the', 'original', 'echo', '.', 'this', 'is', 'the', 'same', 'but', 'shorter', 'and', 'with', 'greater', 'fabric', '/', 'color', 'choices', '.', 'i', 'miss', 'the', 'volume', 'ring', 'on', 'top', ',', 'now', 'it', "'", 's', 'just', 'the', 'plus', '/', 'minus', 'buttons', '.', 'not', 'a', 'big', 'deal', 'but', 'the', 'ring', 'w', 'as', 'comforting', '.', ':', ')', 'other', 'than', 'that', ',', 'well', 'i', 'do', 'like', 'the', 'use', 'of', 'a', 'standard', 'usb', 'charge', '##r', '/', 'port', 'instead', 'of', 'the', 'previous', 'round', 'pin', '.', 'other', 'than', 'that', ',', 'i', 'guess', 'it', 'sounds', 'the', 'same', ',', 'seems', 'to', 'work', 'the', 'same', ',', 'still', 'answers', 'to', 'alexa', '/', 'echo', '/', 'computer', '.', 'so', 'what', "'", 's', 'not', 'to', 'like', '?', ':', ')', '[SEP]']


In [ ]:
MAX_LEN = 64

# input ids

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]


# pad input tokens

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


# create attention masks

attention_masks = []


for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

print(attention_masks[2])

# train and test split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels,
                                                                                    random_state=2018,
                                                                                    test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state=2018,
                                                       test_size=0.1)
# Convert train_masks to a tensor
train_masks = torch.tensor(train_masks)
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)

#convert valiations to tensor
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_inputs = torch.tensor(validation_inputs)


# Convert to TensorDataset
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

# Similar change for validation_masks


'''convert validation dataset to tensordataset and dataloader is to iterate enitre dataset without
loading into memory, dataloader will bundle inputids, attention mask and labels in batches.
and one tensor converts to triples of input ids, attention mask and labels.

(tensor([ 101, 2002, 2939, 1996, 3328, 1012,  102,0 , 0 ....]),
 tensor([1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0]),
 tensor(1))
'''

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=32)

print(train_data[0])

'''

1. Modify pretrainedd BERT to give outputs for classification.
2. training end-to-end suited for our task.
3. Each Toplayers are output types are designed for specific NLP task.
4. BERTForSequenceClassification: Normal bert model with an added single layer ontop for classification for sentence classifier.

Structure of fine tuning model:
1. [the hidden state , number of labels] are passed to outoput of bert plus classification layer
2. vector representing the "score" for grammatical/non grammatical is returned.
3. This is fed to cross entropy loss.

FineTuning process:
1. 95% of bottowm layers are already trained,
2. we need train top layer, with tweeking
3. some times "FREEZE" option is used to freeze bottom layers.
4. or Apply differene learning rates for good quality weights.
5.

'''

from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)



[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(tensor([ 101, 4165, 2307,  999,  999, 2293, 2068,  999,  102,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]), tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# hyperparameters


learning_rate = 2e-5
weight_decoy = 0.01
epsilon = 1e-8

optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon,
                  weight_decay = weight_decoy)

# Start training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create an empty list to store loss values
epoch_losses = []
epochs= 2

for _ in range(epochs):
  total_loss = 0
  model.train()
  tr_loss = 0
  nb_tr_steps = 0
  nb_tr_examples = 0

  for step, batch in enumerate(train_dataloader):
    #batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    optimizer.zero_grad()
    outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)
    # Get the loss from the model output
    loss = outputs.loss
    total_loss += loss.item()
    loss.backward()
    optimizer.step()
    tr_loss += loss.item()
    nb_tr_steps += 1
    nb_tr_examples += b_input_ids.size(0)
    if step % 100 == 0:
      loss_step = tr_loss/nb_tr_steps
      print(f"Training loss after {step}: {loss_step}")

  print(f"Training loss epoch: {epochs}: {tr_loss/nb_tr_steps}")
  # Append the average epoch loss to the list
  epoch_losses.append(tr_loss/nb_tr_steps)

  # validation
  model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  for batch in validation_dataloader:
    #batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      outputs = model(b_input_ids,
                      token_type_ids=None,
                      attention_mask=b_input_mask)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      eval_accuracy += tmp_eval_accuracy
      nb_eval_steps += 1
  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training loss after 0: 0.4581630229949951
Training loss epoch: 2: 0.2135581536992882


NameError: name 'np' is not defined

In [ ]:
model

Model 3

In [ ]:
# %%
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
#from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, Trainer, TrainingArguments


reviews = pd.read_csv("/content/amazon_alexa.tsv",  sep="\t")
pd.set_option("display.max_rows", 5)

print("Setup complete.")

sentences = reviews['verified_reviews'].values

labels = reviews['feedback'].values

# split data set to train and test
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2, random_state=42)


# 2. Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Convert train_sentences to a list of strings before tokenization
train_sentences = train_sentences.astype(str).tolist()
test_sentences = test_sentences.astype(str).tolist()

# Tokenize the dataset
train_encodings = tokenizer(train_sentences, truncation=True, padding=True, max_length=64, return_tensors="pt")
test_encodings = tokenizer(test_sentences, truncation=True, padding=True, max_length=64, return_tensors="pt")
# Convert labels to tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)


# 3. Create a PyTorch Dataset class
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

# 4. Load the pre-trained BERT model for classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


# 5. Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# 6. Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save model checkpoints
    evaluation_strategy="epoch",    # Evaluate after every epoch
    learning_rate=5e-5,             # Learning rate
    per_device_train_batch_size=16, # Batch size for training
    per_device_eval_batch_size=16,  # Batch size for evaluation
    num_train_epochs=3,             # Number of training epochs
    weight_decay=0.01,              # Weight decay for regularization
    logging_dir="./logs",           # Directory for logs
    logging_steps=10,               # Log every 10 steps
    save_steps=500,                 # Save the model every 500 steps
    save_total_limit=2,             # Limit the number of saved checkpoints
    load_best_model_at_end=True,    # Load the best model at the end of training
    save_strategy="epoch",          # Save the model after each step
)

# 7. Initialize the Trainer
trainer = Trainer(
    model=model,                         # The pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=test_dataset,           # Evaluation dataset
    tokenizer=tokenizer,                 # Tokenizer
)

# 8. Train the model
trainer.train()

# 9. Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

path = '/content/drive/MyDrive/Colab Notebooks'
trainer.save_model(path)

# Optional: Save the fine-tuned model
model.save_pretrained(path)
tokenizer.save_pretrained(path)


Setup complete.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


<ipython-input-1-f777ad46976d>:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-1-f777ad46976d>:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss


In [3]:
train_sentences[:1]

array(['Just the perfect cost effective solution.  Thank you Amazon.'],
      dtype=object)